In [1]:
### Colab define cell
FOLDER_PATH = '/content/gdrive/MyDrive/Counterfactual-prototype-main/LORE/LORE-master'

try:
    import google.colab
    RUNNING_ON_COLAB = True
except: 
    RUNNING_ON_COLAB = False

if RUNNING_ON_COLAB: 
    import sys
    from google.colab import drive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials 
    drive.mount('/content/gdrive', force_remount=True)
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    gdrive = GoogleDrive(gauth)
    sys.path.append(FOLDER_PATH)

In [2]:
from utils.prediction import PredictionTypeWrapper, PredictionTypeWrapper, generate_local_predictions, wrap_information
from lore import LOREWrapper
from lore.prepare_dataset import prepare_diabetes_dataset
import lore

from utils import load

import tensorflow as tf
import numpy as np
import pandas as pd

from lore.prepare_dataset import *
from lore.neighbor_generator import *

### Set random seed
seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
### Load data
dataset = load.SelectableDataset.Diabetes
data, balanced_data, X, Y, encoder, scaler, n_features, n_classes, feature_names, target_name = load.load_dataset(dataset)
X_train, Y_train, X_test, Y_test, X_validation, Y_validation = load.load_training_data(dataset)
model = load.load_trained_model_for_dataset(dataset)

Loaded model from disk


In [4]:

diabetes_feature_range = (X_train.min(axis=0), X_train.max(axis=0))
# store all information 
local_data_dict = generate_local_predictions( X_test, Y_test, model, scaler, encoder )
# sorting by different conditions
true_positives,true_negatives, false_positives, false_negatives = wrap_information( local_data_dict )
# get all prediction types
all_predictions = PredictionTypeWrapper(true_positives,true_negatives, false_positives, false_negatives)

In [5]:
lore_wrapper = LOREWrapper.LOREWarpper(model)
diabetes_dataset = prepare_diabetes_dataset('./datasets/diabetes.csv')

In [6]:
X2E = X_test
y2E = lore_wrapper.predict(X2E).flatten()
y2E = np.asarray([diabetes_dataset['possible_outcomes'][i] for i in y2E])

In [7]:
print(diabetes_dataset['label_encoder'][diabetes_dataset['class_name']].classes_)
print(diabetes_dataset['possible_outcomes'])

[0 1]
[1, 0]


In [26]:
diabetes_dataset['columns']

['Outcome',
 'Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

In [15]:
from lore.gpdatagenerator import calculate_feature_values

class_name = diabetes_dataset['class_name']
columns = diabetes_dataset['columns']
discrete = diabetes_dataset['discrete']
continuous = diabetes_dataset['continuous']
features_type = diabetes_dataset['features_type']
label_encoder = diabetes_dataset['label_encoder']
possible_outcomes = diabetes_dataset['possible_outcomes']

diabetes_dataset['feature_values'] = calculate_feature_values(X2E, columns, class_name, discrete, continuous, 1000, True, False)


In [35]:
dfZ, x = dataframe2explain(X2E, diabetes_dataset, 1, lore_wrapper)

In [42]:
x = X2E[4,:]

In [43]:
np.unique(dfZ['Pregnancies'])

array([0])

In [48]:
diabetes_dataset['idx_features']

{0: 'Pregnancies',
 1: 'Glucose',
 2: 'BloodPressure',
 3: 'SkinThickness',
 4: 'Insulin',
 5: 'BMI',
 6: 'DiabetesPedigreeFunction',
 7: 'Age'}

In [44]:
diabetes_dataset['feature_values'][1].shape

(1080,)

In [45]:
diabetes_dataset['feature_values'] 

{0: array([ 0.05882353,  0.05882353,  0.        , ...,  0.12407916,
         0.133779  , -0.00711518]),
 1: array([0.94974874, 0.90452261, 0.52763819, ..., 0.67149076, 0.78548933,
        0.81164853]),
 2: array([0.49180328, 0.        , 0.73770492, ..., 0.61444643, 0.50914006,
        0.46890879]),
 3: array([ 0.23232323,  0.        ,  0.        , ...,  0.33073264,
        -0.08149241,  0.17456991]),
 4: array([1.        , 0.        , 0.        , ..., 0.23221071, 0.02034111,
        0.02605492]),
 5: array([0.4485842 , 0.64530551, 0.44113264, ..., 0.40500143, 0.4871166 ,
        0.63331199]),
 6: array([0.13404711, 0.08436831, 0.04796574, ..., 0.22456815, 0.147034  ,
        0.10089395]),
 7: array([ 0.74509804,  0.39215686,  0.49019608, ...,  0.36726523,
        -0.04772325,  0.18759433])}

In [46]:
idx_record2explain = 0

In [47]:
path_data = './lore/datasets/'
explanation, infos = lore.explain(idx_record2explain, X2E, diabetes_dataset, lore_wrapper,
                            ng_function=genetic_neighborhood,
                                  discrete_use_probabilities=True,
                                  continuous_function_estimation=False,
                                  returns_infos=True,
                                  path=path_data, sep=';', log=False)

/Users/jrhs/miniforge3/envs/tf_mac/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/jrhs/miniforge3/envs/tf_mac/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


OSError: [Errno 8] Exec format error: 'lore/yadt/dTcmd'

In [ ]:
dfX2E = build_df2explain(lore_wrapper, X2E, diabetes_dataset).to_dict('records')
dfx = dfX2E[idx_record2explain]
# x = build_df2explain(blackbox, X2E[idx_record2explain].reshape(1, -1), dataset).to_dict('records')[0]

print('x = %s' % dfx)
print('r = %s --> %s' % (explanation[0][1], explanation[0][0]))
for delta in explanation[1]:
    print('delta', delta)

covered = lore.get_covered(explanation[0][1], dfX2E, diabetes_dataset)
print(len(covered))
print(covered)

print(explanation[0][0][diabetes_dataset['class_name']], '<<<<')

def eval(x, y):
    return 1 if x == y else 0

precision = [1-eval(v, explanation[0][0][diabetes_dataset['class_name']]) for v in y2E[covered]]
print(precision)
print(np.mean(precision), np.std(precision))
